In [14]:
# Torch + HF Transformers
import torch
import transformers
from transformers import AutoTokenizer

# Data Handeling
import pandas as pd
import datasets
from datasets import Dataset

# Text Cleaning
import nltk 
from nltk.stem.porter import *



In [15]:
!nvidia-smi

Sat Feb 26 15:17:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:25:00.0  On |                  N/A |
| 28%   31C    P8     7W / 120W |   3035MiB /  3072MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce GTX 1060 3GB


In [17]:

df = pd.read_csv("../data/labeled_data.csv")
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [18]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet'],
    num_rows: 24783
})

In [20]:
stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()


def preprocess_tweet(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text


In [21]:
preprocess_tweet(" fuck https://shitshow.com")

' fuck '

In [22]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)


def preprocess_function(data):
    processed_tweets = [preprocess_tweet(tweet) for tweet in data["tweet"]]
    return tokenizer(processed_tweets,padding = "max_length",max_length=128,truncation=True)

In [23]:
tokenized_dataset = dataset.map(preprocess_function,batched = True)
tokenized_dataset = tokenized_dataset.rename_column("class","labels")
tokenized_dataset = tokenized_dataset.remove_columns(["hate_speech","offensive_language","neither","tweet","count"])


100%|██████████| 25/25 [00:02<00:00,  9.58ba/s]


In [24]:
tokenized_dataset

Dataset({
    features: ['Unnamed: 0', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 24783
})

In [25]:
train_test_ds = tokenized_dataset.train_test_split(test_size=0.1)
print(train_test_ds)
print(train_test_ds["train"])

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 19826
    })
    test: Dataset({
        features: ['Unnamed: 0', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 4957
    })
})
Dataset({
    features: ['Unnamed: 0', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 19826
})


In [27]:
train_test_ds.save_to_disk("../data/processed_data")

Flattening the indices: 100%|██████████| 5/5 [00:01<00:00,  3.62ba/s]
